In [1]:
%%capture
!pip install transformers 

In [2]:
import numpy as np
import os
from tqdm import tqdm
import copy

import torch
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# model optim
from torch.optim import AdamW, SGD

# lr schedulers
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup

In [3]:
class LangData(Dataset):
    def __init__(self, base_dir, lang, split):
        
        self.split = split
        
        x_file = os.path.join(base_dir, lang+"." + self.split + ".src")
        self.x = self.__read__(x_file)
        
        if self.split != 'test':
            y_file = os.path.join(base_dir, lang+"." + self.split + ".tgt")
            self.y = self.__read__(y_file)
    
    def __read__(self, file_dir):
        
        data = []
        with open(file_dir, 'r') as f:
            for line in f:
                data.append(line.strip())
        return data        
    

    def __len__(self):
        return len(self.x)
    
    
    def __getitem__(self, idx):
        if self.split == "test":
            return self.x[idx]
        return self.x[idx], self.y[idx]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")

In [7]:
# tokenize the text features
def prepare_features_q(data, cache_path, tokenizer, max_len_inp=512,max_len_out=512):
    '''
    tokenize the text features and embed them by the inputed tokenizer
    the features are dump to the cache_path via pickle to avoid re-computation next imte
    '''

    inputs = []
    targets = []
    for q, a, c in tqdm(data):
        input_ = f"context: {c}  answer: {a}" # T5 Input format for QA tasks
        target = f"question: {str(q)}" # Output format we require

    # tokenize inputs
    tokenized_inputs = tokenizer.batch_encode_plus(
      [input_], max_length=max_len_inp,padding='max_length',
      return_tensors="pt" #pytorch tensors
    )
    # tokenize targets
    tokenized_targets = tokenizer.batch_encode_plus(
      [target], max_length=max_len_out,
      padding='max_length',return_tensors="pt"
    )

    inputs.append(tokenized_inputs)
    targets.append(tokenized_targets)

    all_features = {}
    all_features['input'] = inputs
    all_features['target'] = targets
    pkl.dump(all_features, open(cache_path, 'wb')) # dump the features somewhere

In [8]:
def __save_model(model_dir, model, model_type='latest'):

    if model_type == "latest":
        saved_name = 'latest_model.pt'
    else:
        saved_name = 'best_model.pt'

    root_model_path = os.path.join(model_dir, saved_name)
    state_dict = {'weights': model.state_dict(), 
                  'optimizer': model.optimizer.state_dict(), 
                  'scheduler': model.scheduler.state_dict()}
    torch.save(state_dict, root_model_path)
        
        

# Loads the experiment data if exists to resume training from last saved checkpoint.
def __load_experiment(model_dir, model, model_type='latest'):
    
    if model_type == "latest":
        saved_name = 'latest_model.pt'
    else:
        saved_name = 'best_model.pt'

    if os.path.exists(os.path.join(model_dir, 'train.log')):
        # get current epoch
        current_epoch = 0
        with open(os.path.join(model_dir, 'train.log')) as f:
            for line in f:
                current_epoch += 1
        # get the latest model
        state_dict = torch.load(os.path.join(model_dir, saved_name), map_location=device.type)
        model.load_state_dict(state_dict['weights'])
        model.optimizer.load_state_dict(state_dict['optimizer'])
        model.scheduler.load_state_dict(state_dict['scheduler'])
    else:
        current_epoch = 0

    return model, current_epoch



def log(output_dir, log_str, file_name=None):
    if file_name is None:
        file_name = "all.log"
    output_file = os.path.join(output_dir, file_name)
    with open(output_file, 'a') as f:
        f.write(log_str + '\n')


def get_optimizer(model, opt_name, lr, eps): 
    if opt_name == 'Adam':
        return AdamW(model.parameters(), lr=lr, eps=eps)
    elif opt_name == 'SGD':
        return SGD(model.parameters(), lr=lr, eps=eps)
    
    
def get_scheduler(model, scheduler, n_batches, n_epochs, warmup_portion=0.1):
    train_steps = n_epochs*n_batches
    warm_step = int(train_steps*warmup_portion)
    if scheduler == "linear": 
        return get_linear_schedule_with_warmup(model.optimizer, num_warmup_steps=warm_step,num_training_steps=train_steps)
    elif scheduler == "cosine":
        return get_cosine_schedule_with_warmup(model.optimizer, num_warmup_steps=warm_step,num_training_steps=train_steps)


In [9]:
# training loop
def train(model, dataloader_train, n_epochs, model_dir, log_file):

    model.train() # put to train mode
    
    # load current model if exist
    model, current_epoch = __load_experiment(model_dir, model)
    
    all_losses = []
    
    for e in range(current_epoch, n_epochs):

        losses = 0
        for step, data in tqdm(enumerate(dataloader_train), total=len(dataloader_train)):
            
            tokenized_inputs = tokenizer.batch_encode_plus(
              data[0], max_length=64,padding='max_length',
              return_tensors="pt" #pytorch tensors
            )
            # tokenize targets
            tokenized_targets = tokenizer.batch_encode_plus(
              data[1], max_length=64,
              padding='max_length',return_tensors="pt"
            )
            
            input_ids = tokenized_inputs['input_ids'].squeeze()
            target_ids = tokenized_targets['input_ids'].squeeze()
            input_mask = tokenized_inputs['attention_mask'].squeeze()
            target_mask = tokenized_targets['attention_mask'].squeeze()
            labels = copy.deepcopy(target_ids)
            labels[labels == 0] = -100
            
            outputs = model(
                input_ids=input_ids.to(device),
                attention_mask=input_mask.to(device),
                decoder_input_ids=target_ids.to(device),
                decoder_attention_mask=target_mask.to(device), 
                labels = labels.to(device)
            )

            loss = outputs[0]

            model.optimizer.zero_grad() # clear loss
            loss.backward()
            model.optimizer.step()  # backprop to update the weights

            if model.scheduler is not None:
                model.scheduler.step()  # update learning rate schedule 

            # log losses
            loss /= len(dataloader_train) # already average across batch in nn.CrossEntropy
            losses += loss.item()
            
        # output stats
        print(f"Epoch {e}; loss {losses}")
        log(model_dir, "Epoch " + str(e+1) + "; loss " + str(losses), log_file)
        all_losses.append(losses)
        # save model
        __save_model(model_dir, model) # save latest
        if (e > current_epoch and losses < all_losses[-1]):
            __save_model(model_dir, model, model_type='best') # save best model        

In [25]:
def validation(model, dataloader_test, model_dir, log_file):
    
    model, e = __load_experiment(model_dir, model, model_type='latest')
    
    model.eval()
    
    losses = 0
    for step, data in tqdm(enumerate(dataloader_test), total=len(dataloader_test)):

        tokenized_inputs = tokenizer.batch_encode_plus(
              data[0], max_length=64,padding='max_length',
              return_tensors="pt" #pytorch tensors
            )
        # tokenize targets
        tokenized_targets = tokenizer.batch_encode_plus(
          data[1], max_length=64,
          padding='max_length',return_tensors="pt"
        )
            
        input_ids = tokenized_inputs['input_ids'].squeeze()
        target_ids = tokenized_targets['input_ids'].squeeze()
        input_mask = tokenized_inputs['attention_mask'].squeeze()
        target_mask = tokenized_targets['attention_mask'].squeeze()
        labels = copy.deepcopy(target_ids)
        labels[labels == 0] = -100
        
        outputs = model(
            input_ids=input_ids.to(device),
            attention_mask=input_mask.to(device),
            decoder_input_ids=target_ids.to(device),
            decoder_attention_mask=target_mask.to(device),
            labels = labels.to(device)
        )

        loss = outputs[0]

        # log losses
        loss /= len(dataloader_test)
        losses += loss.item()
        
    # output stats
    print(f"Validation loss {losses}")
    log(model_dir, "Validation loss " + str(losses), log_file)

In [16]:
# specify device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 4

checkpoint = 't5_base'
n_epochs = 10
lr = 1e-4
weight_decay = 1e-5
optim = "Adam"
scheduler = "cosine"
log_file = 'train.log'

In [17]:
# model state_dict
model_dir = f"{checkpoint}_e{n_epochs}_lr{lr}_eps{weight_decay}_{optim}_{scheduler}_batch{batch_size}"
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

In [18]:
data = LangData('miniproj1-dataset', 'tar', 'train')
dataloader_train = DataLoader(data, batch_size=batch_size)

In [13]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [19]:
# model optimizer
model.optimizer = get_optimizer(model, optim, lr, weight_decay)

# learning rate scheduler
model.scheduler = get_scheduler(model, scheduler, len(dataloader_train), n_epochs)

In [20]:
type(model.scheduler)

torch.optim.lr_scheduler.LambdaLR

In [21]:
train(model, dataloader_train, n_epochs, model_dir, log_file)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:26<00:00,  6.84it/s]


Epoch 0; loss 3.353860343573615


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:20<00:00,  9.07it/s]


Epoch 1; loss 0.4838574689856614


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:20<00:00,  9.12it/s]


...

Epoch 7; loss 0.016197180237554676


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:20<00:00,  9.04it/s]


Epoch 8; loss 0.01591456891276266


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:20<00:00,  9.10it/s]


Epoch 9; loss 0.014020012310311358


In [22]:
data = LangData('miniproj1-dataset', 'tar', 'dev')
dataloader_test = DataLoader(data, batch_size=batch_size)

In [26]:
validation(model, dataloader_test, model_dir, log_file)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 33.86it/s]

Validation loss 0.01881759754041923


In [27]:
# -----------load trained model -----------------
    
# load latest model with weights computed during training
model, current_epoch = __load_experiment(model_dir, model, model_type='latest')

In [29]:
data = LangData('miniproj1-dataset', 'tar', 'test')
dataloader_test = DataLoader(data, batch_size=batch_size)

In [60]:
# evaluate the model
model.eval()

# context to feed into model
context = "sústima"
inputs = tokenizer(context, return_tensors='pt')
input_ids = inputs['input_ids'].to(device)

# generate questions with context 
outputs = model.generate(input_ids, max_new_tokens=12)

In [61]:
 # decode output (note that question and answer to that question is generated)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
output = question_answer.split(tokenizer.sep_token)
print(len(output[0]))

23


In [66]:
output[0].split('>')[1]

'sssssssssss'

In [69]:
with open('tar.test.output', 'w') as f:
    for i in range(len(data)):
        inputs = tokenizer(data[i], return_tensors='pt')
        input_ids = inputs['input_ids'].to(device)

        # generate questions with context 
        outputs = model.generate(input_ids, max_new_tokens=12)

        # decode output (note that question and answer to that question is generated)
        question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
        question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
        output = question_answer.split(tokenizer.sep_token)
        print(output[0].split('>')[1])
#         f.write(output[0].split('>')[1])
#         f.write('\n')


chachachachachachachachachachacha
bahbahbahbahbahbahbahbahbahbahbah

tóóóóóóóóóó
aaaaaaaaaaa
ttttttttttt

ééééééééééé

hesususususususususus
sfilfilfilfilfilfilfilfilfilfil




aaaaaaaaaaa
kkkkkkkkkkk
uuuuuuuuuuu
ososááááááá

rerererererererererere
rrrrrrrrrrr
rrrrrrrrrrr
rrrrrrrrrrr
ossssssssss
nnnnnnnnnnn
rrrrrrrrrrr

tamamamamamamamamamam
ináááááááááá
k<unk

saaaaaaaaaa
rrrrrrrrrrr

iiiiiiiiiii

tiiiiiiiiii
rimrimrimrimrimrimrimrimrimrimrim
aaaaaaaaaaa
wewewewewewewewewewewe


aaaaaaaaaaa


iiiiiiiiiii
ossssssssss
chonchonchonchonchonchonchonchonchonchonchon

ukukukukukukukukukukuk



imaaaaaaaaaa
siiiiiiiiii
éniéniéniéniéniéniéniéniéniéniéni


bahbahbahbahbahbahbahbahbahbahbah

chochochochochochochochochochocho








waaaaaaaaaa
sssssssssss


kkkkkkkkkkk




nnnnnnnnnnn
aaaaaaaaaaa
eeeeeeeeeee
eeeeeeeeeee
aaaaaaaaaaa
sssssssssss
iiiiiiiiiii

iiiiiiiiiii

rokokokokokokokokokok
aaaaaaaaaaa
